In [ ]:
import sys
sys.path.append("../")

In [ ]:
import json
from pprint import pprint as print_
from collections import OrderedDict
import numpy as np
from numpy.random import RandomState
from tqdm import tqdm_notebook as tqdm

from isanlp_srl_framebank.pipeline_default import PipelineDefault

In [ ]:
random_seed = 41

In [ ]:
roleset44 = {
 'содержание высказывания',
 'говорящий',
 'субъект социального отношения',
 'субъект психологического состояния',
 'содержание действия',
 'агенс',
 'тема',
 'конечная точка',
 'сфера',
 'контрагент',
 'субъект перемещения',
 'причина',
 'субъект поведения',
 'ситуация в фокусе',
 'исходный посессор',
 'субъект физиологической реакции',
 'адресат',
 'пациенс',
 'срок',
 'источник звука',
 'место',
 'признак',
 'потенциальная угроза',
 'субъект ментального состояния',
 'конечный посессор',
 'результат',
 'стимул',
 'субъект восприятия',
 'эффектор',
 'траектория',
 'содержание мысли',
 'пациенс перемещения',
 'каузатор',
 'предмет высказывания',
 'начальная точка',
 'способ',
 'пациенс социального отношения',
 'статус',
 'предмет мысли',
 'цель',
 'потенциальный пациенс',
 'контрагент социального отношения',
 'эталон',
 'признак действия'}

In [ ]:
ARGUMENT_POSTAGS = {
        'NOUN',
        'PRON',
        'ADJ',
        'PROPN'
    }

In [ ]:
def get_roles_pred(lemma, role_annot, part_id):
    ann_sent = role_annot[part_id]
    predicates = {}
    arguments = {}
    for event in ann_sent:
        predicate = {
            'lemma': lemma[part_id][event.pred[0]],
        }
        predicates[event.pred[0]] = predicate
        arguments[event.pred[0]] = []
        for arg in event.args:
            argument = {
                'tag': arg.tag,
                'lemma': lemma[part_id][arg.begin],
                'idx': arg.begin
            }
            arguments[event.pred[0]].append(argument)
            
    return predicates, arguments


def get_example(corpus, ex_number, part_id):
    words = []
    for obj in corpus[ex_number][1][part_id]:
        word = obj['form']
        for symbol in ':;,.!?':
            word = word.replace(' ' + symbol, symbol)
        words.append(word)
        
    return ' '.join(words)


def get_roles_true(annot, corpus, ex_number, part_id):
    predicates = {}
    arguments = {}
    postags = [item for sublist in annot['postag'] for item in sublist]
    for i, obj in enumerate(corpus[ex_number][1][part_id]):
        if 'rank' in obj:
            if obj['rank'] == 'Предикат':
                predicate = {
                    'lemma': obj['lemma']
                }
                predicates[i] = predicate
            else:
                if 'lemma' not in obj:
                    argument = {
                        'lemma': obj['form'],
                        'tag': obj['rolepred1'],
                        'idx': i
                    }
                else:
                    argument = {
                        'lemma': obj['lemma'],
                        'tag': obj['rolepred1'],
                        'idx': i
                    }
                
                argument['postag'] = postags[argument['idx']]
                pred_id = obj['fillpred']
                if pred_id not in arguments.keys():
                    arguments[pred_id] = []
                arguments[pred_id].append(argument)
                
    return predicates, arguments

In [ ]:
def random_predictions(corpus, ppl, n_samples=100):
    np.random.seed(random_seed)
    samples_idxs = np.random.choice(len(corpus), size=n_samples)
    
    texts = [get_example(corpus, ex_num, 0) for ex_num in samples_idxs]
    annotations = [ppl(text) for text in tqdm(texts, desc='Analyzing texts')]
    pred_roles = [get_roles_pred(res['lemma'], res['srl'], 0) for res in annotations]
    
    true_roles = [get_roles_true(annotations[i], corpus, ex_num, 0) for i, ex_num in enumerate(samples_idxs)]
    
    
    repl_roles = {
        'агенс - субъект восприятия' : 'субъект восприятия',
        'агенс - субъект ментального состояния' : 'субъект ментального состояния',
        'результат / цель' : 'результат',
        'место - пациенс' : 'место',
        'говорящий - субъект психологического состояния' : 'субъект психологического состояния'
    }
    
    for role, val in repl_roles.items():
        for pair in true_roles:
            for _, args in pair[1].items():
                for arg in args:
                    arg['tag'] = arg['tag'].replace(role, val)
                    
    return true_roles, pred_roles, texts


def compute_metrics(y_pred, y_true, report_to=sys.stdout, roleset=roleset44, idxmatching=False):
    true_positive = 0
    condition_positive = 0
    predicted_condition_positive = 0
    error_examples = []
    
    print_func = lambda x: print(x, file=report_to)
    
    for i, (true_predicates, true_arguments) in enumerate(y_true):
        print_func(f"Inspecting example {i}")
        print_func(f"Expecting true predicates {true_predicates}")
        print_func(f"Expecting true arguments  {true_arguments}")
        
        pred_predicates, pred_arguments = y_pred[i]

        print_func(f"Got predicted predicates  {pred_predicates}")
        print_func(f"Got predicted arguments   {pred_arguments}")
        
        print_func("-"*60)
        
        for true_pred_idx, true_predicate in true_predicates.items():
            if true_pred_idx in pred_predicates:
                print_func(f"Matched predicate {true_pred_idx} = {true_predicate}")
                
                true = true_arguments[true_pred_idx]
                pred_arguments_i = pred_arguments[true_pred_idx]
                
                true_arguments_i = []
                
                for idx, true_argument in enumerate(true):
                    if true_argument['tag'] in roleset and true_argument['postag'] in ARGUMENT_POSTAGS:
                        true_arguments_i.append(true[idx])
                        
                if true_arguments_i:
                    print_func(f"Expecting arguments  {true_arguments_i}")
                    print_func(f"Got predicted        {pred_arguments_i}")
                    print_func(f"Predicted Condition Positive = {len(pred_arguments_i)}")
                    print_func(f"Condition Positive           = {len(true_arguments_i)}")
                    condition_positive += len(true_arguments_i)
                    condition_positive_i = len(true_arguments_i)
                    predicted_condition_positive += len(pred_arguments_i)

                    true_positive_i = 0

                    error_report = {
                        'example_idx' : i,
                        'predicate': true_predicate,
                        'true_arguments' : true_arguments_i,
                        'predicted_arguments': pred_arguments_i
                    }

                    for j, obj in enumerate(true_arguments_i):
                        true_tag = obj['tag']
                        true_lemma = obj['lemma']
                        true_idx = obj['idx']
                        for obj_pred in pred_arguments_i:
                            if idxmatching:
                                if obj_pred['idx'] == true_idx:
                                    true_positive_i += 1
                            else:
                                if obj_pred['idx'] == true_idx:
                                    true_positive_i += 1

                    print_func(f"True Positive = {true_positive_i}")
                    if true_positive_i != condition_positive_i:
                        error_examples.append(error_report)

                    true_positive += true_positive_i
                
        print_func("="*60)
       
    recall = true_positive/condition_positive
    precision = true_positive/predicted_condition_positive
    
    return {
        'recall': recall,
        'precision': precision,
        'f1': 2 * ((precision*recall)/(precision+recall)),
        'errors': error_examples
    }

In [ ]:
#corpus_name = 'annotated_corpus_fixed+syntaxnet.json'
corpus_path = '../../data/cleared_corpus.json'
with open(corpus_path, 'r', encoding='utf-8') as f:
    corpus = json.load(f)

In [ ]:
import isanlp

In [ ]:
! pip install -U git+https://github.com/tchewik/isanlp_srl_framebank.git

### Metrics of argument extraction quality

In [ ]:
host = 'vmh2.isa.ru'
host2 = 'echistova.isa.ru'
ppl = PipelineDefault(address_morph=(host, 4333),
                       address_syntax=(host, 4334),
                       address_srl=(host2, 4335))

In [ ]:
true_roles, pred_roles, texts = random_predictions(corpus, ppl, n_samples=400)

In [ ]:
log_path = 'log_idx.txt'
results = compute_metrics(y_pred=pred_roles, y_true=true_roles, 
                          report_to=open(log_path, 'w', encoding='utf-8'), 
                          idxmatching=True)

copyres = dict(results)
del copyres['errors']
print_(copyres)

#### Errors 

In [ ]:
import pandas as pd

ff = pd.DataFrame(results['errors'])

In [ ]:
def neatify(text):
    return text.replace(' ,', ',')\
                .replace(' .', '.')\
                .replace(' ) ', ') ')\
                .replace(' ( ', ' (')\
                .replace(' : ', ': ')
                
ff['text'] = ff.example_idx.map(lambda idx: neatify(texts[idx]))

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
ff